In [7]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np  # Importa numpy per la gestione delle trasformazioni

# Percorsi dei file Excel
f1 = "attaccanti_19.xlsx"
f2 = "attaccanti_20.xlsx"
f3 = "attaccanti_21.xlsx"
f4 = "attaccanti_22.xlsx"
f5 = "attaccanti_23.xlsx"

# Colonne da eliminare
colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad', 'Age', 'MP', 'Starts', 'Min', 
                        'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'PrgP',
                        'SoT%', 'G/Sh', 'G/SoT', 'Dist', 'FK', 'G-xG',
                        'Cmp', 'Att', 'Cmp%', 'CrsPA', 'Tkl',
                        'TklW', 'Mid 3rd', 'Att 3rd', 'Mid 3rd.1',
                        'Succ%', 'TotDist', '2CrdY', 'Fld', 'Off',
                        'Crs', 'Won', 'Lost', 'Won%']

# Leggi e unisci i file Excel
df1 = pd.read_excel(f1)
df2 = pd.read_excel(f2).iloc[1:].reset_index(drop=True)
df3 = pd.read_excel(f3).iloc[1:].reset_index(drop=True)
df4 = pd.read_excel(f4).iloc[1:].reset_index(drop=True)
df_unito = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Rimuovi le colonne specificate dal DataFrame unito
df_unito = df_unito.drop(columns=colonne_da_eliminare, errors='ignore')

# Carica il file attaccanti_23.xlsx e rimuovi le stesse colonne
df5 = pd.read_excel(f5).iloc[1:].reset_index(drop=True)
df5 = df5.drop(columns=colonne_da_eliminare, errors='ignore')

# Separare le colonne in input e output
X = df_unito.iloc[:, 2:]  # Tutte le colonne eccetto le prime due
y = df_unito.iloc[:, :2]  # Le prime due colonne

# Pre-elaborazione
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Divisione in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Costruzione della rete neurale
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(y_train.shape[1])
])

# Compilazione del modello
model.compile(optimizer='adam', loss='mean_squared_error')

# Addestramento del modello
model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_split=0.2)

# Carica il nuovo file e pre-processa i dati
df_new = df5  # Usa il DataFrame df5 che abbiamo già elaborato

# Verifica che le colonne siano allineate con il DataFrame di addestramento
X_new = df_new[X.columns]

# Pre-elabora i dati nuovi (normalizza con lo stesso scaler usato per l'addestramento)
X_new_scaled = scaler.transform(X_new)

# Usa il modello per fare previsioni
y_pred = model.predict(X_new_scaled)

# Imposta a 0 tutti i valori predetti che sono minori di 0
y_pred = np.clip(y_pred, 0, None)  # Sostituisci i valori negativi con 0

# Visualizza le previsioni
df_new['Pred_Gls'] = y_pred[:, 0]  # Previsioni per la prima colonna (Gls)
df_new['Pred_Ast'] = y_pred[:, 1]  # Previsioni per la seconda colonna (Ast)

print(df_new[['Gls', 'Ast', 'Pred_Gls', 'Pred_Ast']])



# Calcola separatamente la somma delle previsioni per Gls e Ast
somma_goal_Attesi = df_new['Pred_Gls'].sum()
somma_assist_Attesi = df_new['Pred_Ast'].sum()

# Visualizza la somma
print(f"La somma dei goal ottenuti dalla rete neurale è: {somma_goal_Attesi}")
print(f"La somma degli assist ottenuti dalla rete neurale è: {somma_assist_Attesi}")
somma_goal_assist_Attesi = somma_goal_Attesi + somma_assist_Attesi

# Visualizza la somma
print(f"La somma dei goal e assist ottenuti dalla rete neurale è: {somma_goal_assist_Attesi}")

# Calcola separatamente la somma effettiva dei goal e assist nei dati originali
somma_goal_effettiva = df_new['Gls'].sum()
somma_assist_effettiva = df_new['Ast'].sum()

somma_goal_assist_effettiva = somma_goal_effettiva + somma_assist_effettiva
print(f"La somma dei goal effettivi è: {somma_goal_effettiva}")
print(f"La somma degli assist effettivi è: {somma_assist_effettiva}")
print(f"La somma dei goal e assist effettiva è: {somma_goal_assist_effettiva}")



Epoch 1/1000


C:\Users\Mario\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 19.7585 - val_loss: 14.7654
Epoch 2/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.4106 - val_loss: 8.4048
Epoch 3/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.1341 - val_loss: 6.4482
Epoch 4/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.9449 - val_loss: 4.7650
Epoch 5/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.9922 - val_loss: 3.4946
Epoch 6/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.2625 - val_loss: 3.1252
Epoch 7/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.3189 - val_loss: 3.0936
Epoch 8/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.1279 - val_loss: 2.9028
Epoch 9/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2065 - val_loss: 2.8280
Epoch 10/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.8887 - val_loss: 2.7525
Epoch 11/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.1082 - val_loss: 2.5782
Epoch 12/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2

Epoch 69/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0440 - val_loss: 1.9653
Epoch 70/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0550 - val_loss: 1.8430
Epoch 71/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0694 - val_loss: 1.9279
Epoch 72/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0707 - val_loss: 1.8377
Epoch 73/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9585 - val_loss: 2.0581
Epoch 74/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9419 - val_loss: 1.8580
Epoch 75/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0863 - val_loss: 2.0077
Epoch 76/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0285 - val_loss: 1.8956
Epoch 77/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0693 - val_loss: 1.9646
Epoch 78/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9509 - val_loss: 1.8829
Epoch 79/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9317 - val_loss: 1.8331
Epoch 80/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7610 - val_loss: 2.0606
Epoch 137/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6967 - val_loss: 2.0535
Epoch 138/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8042 - val_loss: 2.1593
Epoch 139/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9133 - val_loss: 1.9820
Epoch 140/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7371 - val_loss: 2.0211
Epoch 141/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7276 - val_loss: 1.9603
Epoch 142/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7032 - val_loss: 1.9798
Epoch 143/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7022 - val_loss: 2.0677
Epoch 144/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8072 - val_loss: 1.9949
Epoch 145/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6870 - val_loss: 2.0513
Epoch 146/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7731 - val_loss: 2.1154
Epoch 147/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4m

...

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0779 - val_loss: 3.9442
Epoch 673/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0696 - val_loss: 3.8613
Epoch 674/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0741 - val_loss: 3.8322
Epoch 675/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0688 - val_loss: 3.9520
Epoch 676/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0708 - val_loss: 3.7635
Epoch 677/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0719 - val_loss: 3.8862
Epoch 678/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0682 - val_loss: 3.8611
Epoch 679/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0716 - val_loss: 3.8140
Epoch 680/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0682 - val_loss: 3.8688
Epoch 681/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0801 - val_loss: 3.8257
Epoch 682/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0811 - val_loss: 3.9377
Epoch 683/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3m

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0674 - val_loss: 3.9255
Epoch 740/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0703 - val_loss: 4.0035
Epoch 741/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0658 - val_loss: 3.9952
Epoch 742/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0623 - val_loss: 3.9700
Epoch 743/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0532 - val_loss: 3.9648
Epoch 744/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0578 - val_loss: 4.0195
Epoch 745/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0683 - val_loss: 3.9637
Epoch 746/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0578 - val_loss: 4.0095
Epoch 747/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0517 - val_loss: 4.0291
Epoch 748/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0500 - val_loss: 4.0384
Epoch 749/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0503 - val_loss: 3.9930
Epoch 750/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3m

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0421 - val_loss: 4.1022
Epoch 807/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0582 - val_loss: 4.1350
Epoch 808/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0638 - val_loss: 3.9722
Epoch 809/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0636 - val_loss: 3.9934
Epoch 810/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0621 - val_loss: 4.0444
Epoch 811/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0544 - val_loss: 3.9521
Epoch 812/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0604 - val_loss: 4.0397
Epoch 813/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0452 - val_loss: 4.0118
Epoch 814/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0531 - val_loss: 4.0885
Epoch 815/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0481 - val_loss: 3.9914
Epoch 816/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0402 - val_loss: 4.0566
Epoch 817/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6m

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0345 - val_loss: 4.1585
Epoch 874/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0412 - val_loss: 4.1505
Epoch 875/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0495 - val_loss: 4.1825
Epoch 876/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0463 - val_loss: 4.1481
Epoch 877/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0450 - val_loss: 4.0944
Epoch 878/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0437 - val_loss: 4.1219
Epoch 879/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0480 - val_loss: 4.1764
Epoch 880/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0401 - val_loss: 4.2006
Epoch 881/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0524 - val_loss: 4.0965
Epoch 882/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0411 - val_loss: 4.1705
Epoch 883/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0452 - val_loss: 4.1688
Epoch 884/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2m

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1149 - val_loss: 4.1725
Epoch 941/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1357 - val_loss: 4.0647
Epoch 942/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2066 - val_loss: 4.3092
Epoch 943/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1052 - val_loss: 4.2205
Epoch 944/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1424 - val_loss: 4.1583
Epoch 945/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1400 - val_loss: 4.0370
Epoch 946/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1442 - val_loss: 3.9369
Epoch 947/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1099 - val_loss: 4.1237
Epoch 948/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0849 - val_loss: 4.0199
Epoch 949/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0566 - val_loss: 4.0885
Epoch 950/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0548 - val_loss: 4.1440
Epoch 951/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3m